# Libraries

In [ ]:
!pip install catboost
!pip install vecstack

In [ ]:
from catboost import CatBoostRegressor
import pandas as pd
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.model_selection import KFold
from vecstack import StackingTransformer
import os
from xgboost import XGBRegressor

# Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
text_features = [
  "title",
  "retraining_condition",
  "responsibilities",
  "requirements_required_certificates",
  "requirements_qualifications",
  "additional_info",
"career_perspective",
"education_requirements_speciality",
"id_hiring_organization",
"job_benefits_other_benefits",
"job_location_additional_address_info",
"job_location_address"
]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
TEXT_EMBEDINGS = "/content/drive/MyDrive/payroll/"
embedding_names = os.listdir(TEXT_EMBEDINGS + "svd_text_embeddings")
embedding_names

In [ ]:
from tqdm import tqdm
import pickle

embedding_mappers = []

for filename in tqdm(embedding_names):
    with open(f"{TEXT_EMBEDINGS}svd_text_embeddings/{filename}", "rb") as fin:
        unique_embeddings = pickle.load(fin)
        embedding_mappers.append(unique_embeddings)
    

In [ ]:
def clean_sentence(sentence):
    return ''.join(map(lambda c: c if c.isalpha() else ' ', sentence.lower()))

In [ ]:
X = pd.read_csv("/content/drive/MyDrive/payroll/ready_data/X_train_encoded_2.csv", index_col=0)

In [ ]:
import gc
import numpy as np

for i in range(len(embedding_names)):
    feature = embedding_names[i][:-4]
    mapper = embedding_mappers[i]
    X[feature] = X[feature].apply(clean_sentence)

    columns=[f"{feature}_{i}" for i in range(8)]

    data=[]
    for el in tqdm(X[feature]):
        if el not in mapper:
            data.append(np.array([0, 0, 0, 0, 0, 0, 0, 0]))
        else:
            data.append(np.expand_dims(mapper[el], axis=0))
            
    data = np.vstack(data)
    embeddings = pd.DataFrame(columns=columns, data=data)
    X = X.join(embeddings)
    del embeddings
    gc.collect()

In [ ]:
for feature in text_features:
    del X[feature]

gc.collect()

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/payroll/ready_data/X_test_encoded_2.csv", index_col=0)

In [ ]:
test

In [ ]:
import gc
import numpy as np

for i in range(len(embedding_names)):
    feature = embedding_names[i][:-4]
    mapper = embedding_mappers[i]
    test[feature] = test[feature].apply(clean_sentence)

    columns=[f"{feature}_{i}" for i in range(8)]

    data=[]
    for el in tqdm(test[feature]):
        if el not in mapper:
            data.append(np.array([0, 0, 0, 0, 0, 0, 0, 0]))
        else:
            data.append(np.expand_dims(mapper[el], axis=0))
            
    data = np.vstack(data)
    embeddings = pd.DataFrame(columns=columns, data=data)
    test = test.join(embeddings)
    del embeddings
    gc.collect()

In [ ]:
for feature in text_features:
    del test[feature]

gc.collect()

In [ ]:
y = pd.read_csv("/content/drive/MyDrive/payroll/ready_data/y_train.csv", index_col=0)

In [ ]:
y.isnull().sum()

# Delete nan features

In [ ]:
for column in X.columns:
    if X[column].dtype == "object":
        print(column)

In [ ]:
del X["std_profession_mean_salary"]
del X["std_year_quarter_mean_salary"]

In [ ]:
del test["std_profession_mean_salary"]
del test["std_year_quarter_mean_salary"]

## Best features

In [ ]:
import os
IMPORTANCES_PATH = "/content/drive/MyDrive/payroll/feature_importances/catboost/"

folds = os.listdir(IMPORTANCES_PATH)

In [ ]:
importances = [pd.read_csv(IMPORTANCES_PATH + fold) for fold in folds]
importances

In [ ]:
mean_importance = importances[0]
for i in range(1, 6):
    mean_importance["Importances"] += importances[i]["Importances"]

mean_importance["Importances"] = mean_importance["Importances"] / 4

In [ ]:
mean_importance

In [ ]:
valid_features = []
for feature in mean_importance["Feature Id"]:
    if feature == "random":
        break
    valid_features.append(feature)

In [ ]:
valid_features

In [ ]:
X = X[valid_features]
test = test[valid_features]

## Обучим мета-модель

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
# from xgboost import XGBRegressor

kf = KFold(n_splits=6, random_state=42, shuffle=True)
kf.get_n_splits(X)

print(kf)
i = 0
for train_index, test_index in kf.split(X):
    if i in [0, 1]:
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model = CatBoostRegressor(
                                iterations=35000,
                                learning_rate=0.012,
                                depth=11,
                                l2_leaf_reg=4,
                                grow_policy="SymmetricTree",
                                # model_size_reg=None,
                                loss_function='MAE',
                                eval_metric="MAE",
                                verbose=1000,
                                random_strength=2,
                                random_state=42,
                                task_type="GPU",
                                # objective="MSE",
        )  
        
        model.fit(X_train, np.log(y_train["base_salary_max"] + 1), eval_set=(X_test, np.log(y_test["base_salary_max"] + 1)), use_best_model=True)

        preds = np.exp(model.predict(X_test)) - 1
        score = mean_absolute_error(y_test["base_salary_max"], preds)
        print(score)
        
        with open(f'/content/drive/MyDrive/payroll/stacked_preds/max_catboost/max_catboost_fold_{i}_{score}.npy', 'wb') as f:
            np.save(f, preds)
        model.save_model(f"/content/drive/MyDrive/payroll/stacked_models/max_catboost/max_catboost_fold_{i}_{score}.pkl")
    i += 1

# Prediction

In [ ]:
base_stack_path = '/content/drive/MyDrive/payroll/stacked_models/passthrough_catboost/'
models = os.listdir(base_stack_path)

In [ ]:
models

In [ ]:
all_preds = []
for model in tqdm(models):
    estimator = CatBoostRegressor(
                      iterations=30000,
                      learning_rate=0.001,
                      depth=11,
                      l2_leaf_reg=4,
                      # model_size_reg=None,
                      loss_function='MAE',
                      eval_metric="MAE",
                      verbose=1000,
                      random_strength=2,
                      # random_state=42,
                      task_type="GPU",
                      # objective="MSE",
        )  
    estimator.load_model(base_stack_path + model)

    preds = np.exp(estimator.predict(test)) - 1
    
    all_preds.append(preds)


In [ ]:
all_preds

In [ ]:
prediction = all_preds[0]
for i in range(1, 6):
    prediction += all_preds[i]
    
prediction = prediction / 6
prediction

In [ ]:
submission = pd.read_csv(f'/content/drive/MyDrive/payroll/data/sample_submission.csv', index_col=0)

In [ ]:
submission["mean_salary"] = prediction
submission

In [ ]:
submission.to_csv("passthrough_submission.csv")